### Preprocess training data

This is a helper script to preprocess training data that was generated with GIMP. GIMP outputs usually don't have perfect edges - they are always slightly blurry, even if you pick the hardest brush. This script changes that and ensures the labels actually touch.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
import skimage
from skimage.transform import downscale_local_mean
import tqdm
from skimage.morphology import disk
import numpy as np

# lists of paired raw and manually labeled data
path_raw = (
            r"annas_labels/C61 8Mo ON67-0923 L5drvr_1_raw.png",
            r"annas_labels/C61 8Mo ON67-0923 L5drvr_2_raw.png",
            r"annas_labels/C61 8Mo ON67-0923 NSNF_raw.png",
            r"annas_labels/C61 8Mo ON67-0932 L5drvr_raw.png",
            r"annas_labels/C61_P18_272-6wt_drvrL5_01_raw.png",
            r"annas_labels/C61_P18_273-1tg_drvrL5_01_raw.png",
            r"annas_labels/C61_P18_273-5 tg_NsNf_01_raw.png",
            )

path_seg = (
            r"annas_labels/C61 8Mo ON67-0923 L5drvr_1_layer.png",
            r"annas_labels/C61 8Mo ON67-0923 L5drvr_2_layer.png",
            r"annas_labels/C61 8Mo ON67-0923 NSNF_layer.png",
            r"annas_labels/C61 8Mo ON67-0932 L5drvr_layer.png",
            r"annas_labels/C61_P18_272-6wt_drvrL5_01_layer.png",
            r"annas_labels/C61_P18_273-1tg_drvrL5_01_layer.png",
            r"annas_labels/C61_P18_273-5 tg_NsNf_01_layer.png",
            )

# output directory
path_results = r"annas_labels/processed"


In [ ]:
for i in tqdm.tqdm(range(len(path_raw))):
    raw = cv2.imread(path_raw[i])
    seg = cv2.imread(path_seg[i])
    #BGR
    #get thresholded binary images
    seg_blue = seg[:,:,0]>50 # myelin
    seg_green = seg[:,:,1]>50 # low density myelin
    seg_red = seg[:,:,2]>20  # fibers

    # # dilate green, restrict to blue
    # seg_green = ndi.binary_dilation(seg_green,structure=disk(2))
    # seg_green[seg_blue==True]=0

    # dilate blue, restrict to red and green
    seg_blue = ndi.binary_dilation(seg_blue,structure=disk(2))
    seg_blue[seg_red==True]=0
    seg_blue = seg_blue+seg_green

    # dilate red, restrict to blue and green
    seg_red = ndi.binary_dilation(seg_red,structure=disk(2))
    seg_red[seg_blue==True]=0
    # seg_red[seg_green==True]=0

    raw = skimage.exposure.equalize_adapthist(raw,clip_limit=0.01,kernel_size=127)
    # raw_rgb_png = cv2.merge((downscale_local_mean(raw[:,:,0],1),  #R
    #                          downscale_local_mean(raw[:,:,1],1),    #G
    #                          downscale_local_mean(raw[:,:,2],1)))   #B
    plt.imshow(raw)
    # combine back together to RGB
    seg_final = cv2.merge((downscale_local_mean(seg_red,1),  #R
                           downscale_local_mean(np.zeros_like(seg_red),1),#G
                           downscale_local_mean(seg_blue,1)))#B
    skimage.io.imsave(path_results + rf"/AS_labels_{i}.png", (seg_final*255).astype('uint8'))
    skimage.io.imsave(path_results + rf"/AS_raw_{i}.png", (raw*255).astype('uint8'))